In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import notebook
E = notebook.nbextensions.EnableNBExtensionApp()
E.enable_nbextension('usability/python-markdown/main')
import pandas as pd
from scipy.optimize import curve_fit
import markdown
import sys
sys.path.append('/Users/vs/Dropbox/Python')
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord

bigfontsize=20
labelfontsize=16
tickfontsize=16
sns.set_context('talk')
plt.rcParams.update({'font.size': bigfontsize,
                     'axes.labelsize':labelfontsize,
                     'xtick.labelsize':tickfontsize,
                     'ytick.labelsize':tickfontsize,
                     'legend.fontsize':tickfontsize,
                     })

# Matching the Hipparcos and Tycho catalogues to the SMHASH/CRRP observations




## Hipparcos RR Lyrae

* Hipparcos main catalogue - http://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=I/239/hip_main
* Hipparcos variability annex - http://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=I/239/hip_va_1

To get the positions of the RRL out of this catalogue need to match the two up. The variability annex doesn't have RA, Dec.

Can match the two catalogues using TAPVizier
* Uses ADQL - Like SQL
* Need to match on HID
* Want HID, RA, Dec, Type, Period



ADQL query to match the catalogues:

```sql
SELECT "I/239/hip_main".HIP, "RA(ICRS)", "DE(ICRS)", "VarType", "VarName", "I/239/hip_va_1"."Period"
FROM "I/239/hip_main", "I/239/hip_va_1"
WHERE "I/239/hip_main".HIP="I/239/hip_va_1".HIP AND "I/239/hip_va_1".VarType LIKE 'RR%'
```

Output saved to ~/Dropbox/SMHASH/Hipparcos_Comparison/Hipparcos_RRL

## Tycho RR Lyrae

Maintz (2005) claims many IDs of RRL in Tycho 2 catalogue are wrong. Match that catalogue with Tycho 2 to get list of RRL that appear in Tycho.

Maintz catalogue: J/A+A/442/381/table1

Use Cat='T' to select those in Tycho

Output saved to ~/Dropbox/SMHASH/Hipparcos_Comparison/Tycho_RRL

## Find the unique stars

Some of these stars are in both catalogues. Need to filter the catalogues so it's a unique list.

In [11]:
hipparcos = pd.read_csv("Hipparcos_RRL", sep='|')
hipparcos.rename(columns=lambda x: x.strip(), inplace=True) ## Stripping the whitespace out of the column names
hipparcos.rename(columns = {'RA(ICRS)':'RA', 'DE(ICRS)':'Dec'}, inplace=True) ## Getting rid of the (ICRS)
hipparcos = hipparcos.assign(RRName = lambda x: (x.VarName.str.replace('_', '')))

tycho = pd.read_csv("Tycho_RRL", sep='|')
tycho.rename(columns=lambda x: x.strip(), inplace=True)
tycho.rename(columns = {'RAJ2000' : 'RA', 'DEJ2000' : 'Dec', 'VType' : 'VarType'}, inplace=True)
tycho = tycho.assign(RRName = lambda x: (x.Star.str.replace(' ', '')))



In [12]:
hipparcos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186 entries, 0 to 185
Data columns (total 7 columns):
HIP        186 non-null int64
RA         186 non-null float64
Dec        186 non-null float64
VarType    186 non-null object
VarName    186 non-null object
Period     186 non-null object
RRName     186 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 11.6+ KB


In [13]:
tycho.head()

,RA,Dec,Star,VarType,Period,RRName
0,353.002946,35.196908,DM And,RRab,0.630389,DMAnd
1,16.294612,34.218400,DR And,RRab,0.563118,DRAnd
2,5.184192,40.828000,OV And,RRab,0.470581,OVAnd
3,245.214571,-71.671039,BS Aps,RRab,0.582556,BSAps
4,330.481325,-5.600711,TZ Aqr,RRab,0.571194,TZAqr


Match the two catalogues to see which stars appear in both

In [14]:
hipparcos.head()

,HIP,RA,Dec,VarType,VarName,Period,RRName
0,226,0.700312,-24.945255,RRAB,RU_Scl,0.4933470,RUScl
1,320,1.021268,-16.997641,RRAB,UU_Cet,0.6060830,UUCet
2,1222,3.809727,60.340474,RRAB,V363_Cas,0.5465650,V363Cas
3,1878,5.929558,29.401053,RRAB,SW_And,0.4422620,SWAnd
4,2655,8.409568,-15.487302,RRAB,RX_Cet,0.5736850,RXCet


In [15]:
matched = pd.merge(left=hipparcos, right=tycho, how='left', left_on='RRName', right_on='RRName')

In [16]:
len(tycho) + len(hipparcos)

295

**None of the Tycho RRL appear in the Hipparcos RRL catalogue if you match by name. Might miss a couple that way, but this is just an estimate. I can do a proper calculation if we need it.**

# Total number of Hipparcos and Tycho RR Lyrae variables = 295#

# Matching the catalogue to the Spitzer observations

In [17]:
hiptyc = pd.concat([hipparcos, tycho], axis=0)

In [18]:
hiptyc.head()

,Dec,HIP,Period,RA,RRName,Star,VarName,VarType
0,-24.945255,226,0.4933470,0.700312,RUScl,NaN,RU_Scl,RRAB
1,-16.997641,320,0.6060830,1.021268,UUCet,NaN,UU_Cet,RRAB
2,60.340474,1222,0.5465650,3.809727,V363Cas,NaN,V363_Cas,RRAB
3,29.401053,1878,0.4422620,5.929558,SWAnd,NaN,SW_And,RRAB
4,-15.487302,2655,0.5736850,8.409568,RXCet,NaN,RX_Cet,RRAB


Combined data set is now **hiptyc** data frame.

Useful columns are RA, Dec, RRName, VarType, Period

Star, VarName, HIP not defined for all stars.

# Searching for the HipTyc objects in the Spitzer Heritage Archive
* Want to check if we observed thses objects as part of CRRP or SMHASH
* Can check for two things:
  1. Have they been observed at all by Spitzer?
  2. Have they been observed specifically by us?



Have the HipTyc stars been observed at all by Spitzer?
-----------------------------------------------------------------------

* SHA can take a list as input:
  1. Send the catalogue to an output text file
  2. Give this as input to the SHA
  3. Set a small search radius (1 arcmin - want to make sure they're on the frame)
  4. ???
  5. PROFIT

In [19]:
hiptyc.to_csv('hiptyc.csv', columns=('RRName', 'RA', 'Dec'), header=('NAME', 'RA', 'DEC'), index=False)

Used this as input to the SHA search with 1 arcmin search radius. Saved results to hiptyc_obs_by_spitzer.ipac

Now going to search this to find unique targets.

In [25]:
spitzer_ipac = Table.read('hiptyc_obs_by_spitzer.ipac', format='ipac')

In [26]:
spitzer = spitzer_ipac.to_pandas()

In [27]:
spitzer.head()

,Search_Tgt,Target_name,RA(J2000),Dec(J2000),Instrument/Mode,AORKEY,AOR_label,progid,PI
0,0h02m48.0749s 0h02m48.0749s,RU_Scl,0.7,-24.945278,IRAC Map PC,47289600,RU_Scl_epoch5,90002,"Freedman, Wendy"
1,0h02m48.0749s 0h02m48.0749s,RU_Scl,0.7,-24.945278,IRAC Map PC,47291136,RU_Scl_epoch2,90002,"Freedman, Wendy"
2,0h02m48.0749s 0h02m48.0749s,RU_Scl,0.7,-24.945278,IRAC Map PC,47288576,RU_Scl_epoch7,90002,"Freedman, Wendy"
3,0h02m48.0749s 0h02m48.0749s,RU_Scl,0.7,-24.945278,IRAC Map PC,47290112,RU_Scl_epoch4,90002,"Freedman, Wendy"
4,0h02m48.0749s 0h02m48.0749s,RU_Scl,0.7,-24.945278,IRAC Map PC,47290624,RU_Scl_epoch3,90002,"Freedman, Wendy"


Now have the Spitzer search results in a pandas dataFrame!

Each HipTyc RRL can be matched to multiple observations. The matched HipTyc star is given in Search_Tgt

* Need to find how many unique values there are in Search_Tgt column
* Then see who observed them
 - How many are observed by us (i.e. have time series data)
 - How many were observed by other programs (may be single epoch)
 - Which ones are both? Are there ones that could add in extra data points from archival data?
* Can then match back to HipTyc dataFrame to get final catalogue of observed targets.

In [28]:
spitzer['Tgt'], spitzer['Scrap'] = zip(*spitzer['Search_Tgt'].apply(lambda x: x.split(' ', 1)))
del spitzer['Scrap']
del spitzer['Search_Tgt']
s = spitzer.groupby(['Tgt'])
len(s)

70

## Total number of HipTyc stars observed by Spitzer in any program = 70

In [29]:
wlf_obs = spitzer.loc[spitzer['PI']=='Freedman, Wendy']
kvj_obs = spitzer.loc[spitzer['PI']=='Johnston, Kathryn']

In [31]:
wlf_targets = wlf_obs.groupby(['Tgt'])
n_wlf = len(wlf_targets.groups)
kvj_targets = kvj_obs.groupby(['Tgt'])
n_kvj = len(kvj_targets.groups)
n_wlf, n_kvj

(51, 0)

## Number of HipTyc stars observed by CRRP: 51
## Number of HipTyc stars observed by SMHASH: 0

# Which stars were observed by CRRP?

* Need to drop duplicates in the row index from the wlf_obs dataFrame, then match to hiptyc dataFrame

In [32]:
wlf_obs['Search_Tgt_2'] = wlf_obs.Tgt
wlf_obs = wlf_obs.drop_duplicates(cols='Search_Tgt_2', take_last=True)
del wlf_obs['Search_Tgt_2']
wlf_obs.sort_index(inplace=True)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: the 'cols' keyword is deprecated, use 'subset' instead
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## The following table lists the fields that contain Hipparcos or Tycho RR Lyrae variables. 

**Note:** There is an LMC field in here. It was a 2s exposure, so the RRL is probably not saturated.

The relavent fields here are Target_name, progid, and PI if you want to look up the data. AOR label is only going to give you that specific epoch. Every star in this list has at least 24 epochs. 

In [33]:
wlf_obs

,Target_name,RA(J2000),Dec(J2000),Instrument/Mode,AORKEY,AOR_label,progid,PI,Tgt
23,RU_Scl,0.700000,-24.945278,IRAC Map PC,47397632,RU_Scl_epoch14,90002,"Freedman, Wendy",0h02m48.0749s
47,SW_And,5.929167,29.400833,IRAC Map PC,47218688,SW_And_epoch16,90002,"Freedman, Wendy",0h23m43.0939s
72,RU_Psc,18.608333,24.415556,IRAC Map PC,47402496,RU_Psc_epoch22,90002,"Freedman, Wendy",1h14m25.9772s
96,XX_And,19.362500,38.950556,IRAC Map PC,47421696,XX_And_epoch1,90002,"Freedman, Wendy",1h17m27.3702s
123,AM_Tuc,19.625000,-67.917778,IRAC Map PC,47590656,AM_Tuc_epoch1 xc1,90002,"Freedman, Wendy",1h18m30.6120s
147,RR_Cet,23.033333,1.341667,IRAC Map PC,47285248,RR_Cet_epoch12,90002,"Freedman, Wendy",1h32m08.1665s
171,SV_Scl,26.245833,-30.059167,IRAC Map PC,47486464,SV_Scl_epoch24,90002,"Freedman, Wendy",1h44m59.6640s
195,CS_Eri,39.270833,-42.963333,IRAC Map PC,47338496,CS_Eri_epoch8,90002,"Freedman, Wendy",2h37m05.6827s
219,X_Ari,47.125000,10.445833,IRAC Map PC,47330816,X_Ari_epoch5,90002,"Freedman, Wendy",3h08m30.8450s
243,SV_Eri,47.966667,-11.353889,IRAC Map PC,47446272,SV_Eri_epoch16,90002,"Freedman, Wendy",3h11m52.0990s


Now will do a check to see what has been observed by other programs


In [34]:
other_obs = spitzer.loc[spitzer['PI']!='Freedman, Wendy'] ## Any observer except Wendy

In [37]:
other_targets = other_obs.groupby(['Tgt'])
n_other = len(other_targets.groups)
n_other

24

## Total of 24 RRL in HipTyc that have been observed by other programs, but NOT by us.

In [38]:
other_obs['Search_Tgt_2'] = other_obs.Tgt
other_obs = other_obs.drop_duplicates(cols='Search_Tgt_2', take_last=True)
del other_obs['Search_Tgt_2']
other_obs.sort_index(inplace=True)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: the 'cols' keyword is deprecated, use 'subset' instead
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [39]:
other_obs

,Target_name,RA(J2000),Dec(J2000),Instrument/Mode,AORKEY,AOR_label,progid,PI,Tgt
48,U20027_2M0033-1521,8.349417,-15.358583,IRAC Map,22138368,IRAC-30-U20027_2M0033-1521,284,"Cruz, Kelle",0h33m38.2963s
394,DLS-L1,140.216667,30.000000,IRAC Map,17645312,DLS-L1,30478,"Fazio, Giovanni",9h19m06.0352s
491,SDSSJ113246.50+120901.7,173.193750,12.150556,IRAC Map,23767552,IRACfour - 0016,40517,"Meisenheimer, Klaus",11h33m03.7646s
614,NGC4236,184.175417,69.462778,IRAC Map,5504768,IRAC-N4236 - A,159,"Kennicutt, Robert",12h17m46.6736s
703,NGC5311,207.233750,39.985250,IRAC Map PC,31088640,S4G-NGC5311B,61063,"Sheth, Kartik",13h48m15.9375s
728,Bootes II,209.500000,12.850000,IRAC Map PC,42317568,IRACPC-0000 - Bootes II,80063,"Boyer, Martha L",13h58m46.3367s
801,RXJ1518.9-4050 - clus,229.720000,-40.848056,IRAC Map,5666048,EVANS-WTTS-IRAC - RXJ1518.9-4050 - clus,173,"Evans, Neal",15h22m25.3857s
826,NN Ser,238.234167,12.913111,IRAC Map,17725696,NNSer_irac,30070,"Brinkworth, Carolyn S",15h53m31.0547s
875,SDSS J1619+4526,244.992708,45.442000,IRAC Map,26694912,IRAC-0005,50681,"Fan, Xiaohui",16h20m04.0796s
946,357.13525GLIMPSE,264.672590,-31.378248,IRAC Map,14326272,l357.13525-00.01535.20050929,20201,"Churchwell, Ed",17h40m48.4790s


In [40]:
RA_Dec = SkyCoord(ra=(hiptyc.RA)*u.degree, dec=(hiptyc.Dec)*u.degree, frame='icrs')

In [41]:
hiptyc = hiptyc.assign(ra_hours = lambda x: (RA_Dec.ra.to_string('h')))